<a href="https://colab.research.google.com/github/JapiKredi/RAG_assignment_Research_papers/blob/main/RAG_assignment_Research_papers_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment "Course 3 Mr.HelpMate"
### Learner: Jasper Bongers
#### Option 1: Build Your Own Project (BYOP)

- I a building a generative AI system that is reading 5 popular Research Papers in Natural Language Understanding.
- I am using multiple Large language Model Embeddings to translate the text into vector embeddings.
- I am using multiple Large Language Foundation modls. I am using OpenAi, LLaMa2 and other models as well.
- I am using multiple LLM transformer-based models in order to process the research papers and finetune the foundation language models.

# <font color = green> Solution 1


## 1. <font color = red> Install and Import the Required Libraries

Please run the following code on a GPU, as running it on a CPU can create a crash

In [ ]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.4 MB/s 

In [ ]:
!pip install accelerate
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.2 MB/s eta 0:00:00


In [ ]:
import accelerate
import bitsandbytes

In [ ]:
# Import all the required Libraries
import openai
import pypdf
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import chromadb

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from sentence_transformers import CrossEncoder, util

In [ ]:
!pip show langchain

Name: langchain
Version: 0.1.4
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## 2. <font color = red> Read, Process, and Chunk the PDF Files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Set the API key
filepath = "/content/drive/My Drive/GenerativeAI/MateAI/"

with open(filepath + "Jasper_OpenAI_API_Key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = openai.api_key

## Load multiple and process documents

In [ ]:
# Load and process the text files
loader = DirectoryLoader("/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/", glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [ ]:
len(documents)

346

## Splitting, Chunking of text

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

1419

In [ ]:
texts[10]

Document(page_content='1024) since we break batches into multiple updates. See Table 1 in the appendix for exact numbers.\nNonoverlapping Inference To train on or evaluate a sequence longer than Ltokens, it is typical\nto segment the sequence into L-length subsequences and train on or evaluate them independently.\nUnless otherwise stated, we use nonoverlapping inference to report perplexity scores.\nExtrapolation During Inference Formally, the functions that deﬁne a transformer layer are ag-\nnostic to input length;3they map from some arbitrary, unﬁxed number of input vectors to the same\nnumber of output vectors. When transformers are applied to data that is inherently sequential, like\ntext, positional information is injected into the inputs in various ways.\nVaswani et al. (2017) discussed two options for embedding positions into vectors to be added to word\nembeddings: learning embeddings for speciﬁc positions and unlearned sinusoidal embeddings. They', metadata={'source': '/conten

## Embedding the text via regular HuggingFace process

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


### Creating Chroma Vector Database

In [ ]:
# Define the path where chroma collections will be stored
chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = chroma_data_path

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

## 4. <font color = red> Semantic Search

In this section, we will perform a semantic search of a query in the collections embeddings to get several top semantically similar results.

In [ ]:
## Make a retriever
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

In [ ]:
# Get a relevant query
query = "What is Flash attention?"

In [ ]:
docs = retriever.get_relevant_documents("What is Flash attention?")

In [ ]:
retriever.search_type

'similarity'

## Make a chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


## 6. Retrieval Augmented Generation

Now that we have the final top search results, we can pass it to an GPT 3.5 along with the user query and a well-engineered prompt, to generate a direct answer to the query along with citations, rather than returning whole pages/chunks.

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# Querry 1: What is Flash attention?
query = "What is Flash attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




FlashAttention is a new attention algorithm that uses techniques such as tiling and storing the softmax
normalization factor to reduce memory accesses and improve performance in computing attention.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [ ]:
# Querry 2: What is self attention?
query = "What is self-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Self-attention is a mechanism used in neural networks, specifically in Transformer models, to capture
relationships between different input tokens or elements. It involves computing a representation for each
input element by considering its relationship with all other elements in the input sequence. This allows the
model to learn dependencies and long-range relationships between elements, making it useful for tasks such as
natural language processing and sequence prediction. However, self-attention can be computationally expensive
and memory-intensive for long sequences, leading to the development of approximate attention methods.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [ ]:
# Querry 3: What is multi head attention?
query = "What is multi head attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Multi head attention is a computation used in deep learning models, specifically in Transformers, that is
highly memory-intensive. It involves touching the GPU memory multiple times and can be parallelizable across
multiple GPUs. The IO-aware approach aims to optimize this computation and potentially extend it to other
modules in the deep network.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [ ]:
# Querry 4: What does IO-aware mean?
query = "What does IO-aware mean?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 IO-aware means being conscious of input/output operations in order to optimize performance and efficiency.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [ ]:
# Querry 5: What is tiling in flash-attention?
query = "What is tiling in flash-attention?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Tiling in flash-attention is a technique used to reduce the amount of HBM accesses in the attention
algorithm. It involves breaking down the input matrix into smaller tiles and computing attention on each tile
separately, reducing the overall number of HBM accesses needed.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Flash-attention.pdf


In [ ]:
# Querry 6: What tools can be used with toolformer?
query = "What tools can be used with toolformer?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Toolformer can use external tools through simple APIs in a self-supervised way, requiring only a few
demonstrations for each API.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/toolformer.pdf


In [ ]:
# Querry 7: What are the best retrieval augmentations for LLMs?
query = "What are the best retrieval augmentations for LLMs?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Dense and sparse retrievers.


Sources:
/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/research_articles/Augmenting LLMs Survey.pdf


## Deleting the Chroma Vector Database

In [ ]:
# Define the path where chroma collections will be stored
# chroma_data_path = '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# !zip -r db.zip ./db
!zip -r ChromaDB_Data.zip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

In [ ]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data'

## Starting again loading the db

restart the runtime

In [ ]:
!unzip db.zip
!unzip '/content/drive/My Drive/GenerativeAI/MateAI/rag_assignment/ChromaDB_Data.zip'